In [2]:
import numpy as np
import glob
import os
from models.JetPointNet import PointNetSegmentation


def load_data_from_npz(npz_file):
    data = np.load(npz_file)
    feats = data['feats']  # Shape: (num_samples, 859, 6)
    frac_labels = data['frac_labels']  # Shape: (num_samples, 859)
    tot_labels = data['tot_labels']  # Shape: (num_samples, 859)
    tot_truth_e = data['tot_truth_e']  # Shape: (num_samples, 859) (This is the true total energy deposited by particles into this cell)
    return feats, frac_labels, tot_labels, tot_truth_e

# Setup
os.environ['CUDA_VISIBLE_DEVICES'] = ""  # Disable GPU
abs_model_path = "saved_model/PointNetModel.keras"

TEST_DIR = '/data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test'

abs_model = PointNetSegmentation(num_points=859, num_classes=1)
abs_model.load_weights(abs_model_path)

npz_files = glob.glob(os.path.join(TEST_DIR, '*.npz'))



2024-04-08 17:07:47.261379: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9360] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 17:07:47.261634: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 17:07:47.261747: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1537] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 17:07:47.271210: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-08 17:07:49.859646: E tensor

In [7]:
# Initialize metrics
frac_mae_metrics = []
abs_mae_metrics = []

# Process each file
for npz_file_idx, npz_file in enumerate(npz_files):
    feats, frac_labels, tot_labels, tot_truth_e = load_data_from_npz(npz_file)
    current_energies = feats[:, :, 4]  # Energy is at index 4

    # Predictions
    abs_predicted = np.squeeze(abs_model.predict(feats), axis=-1)

    # Compute predicted energies
    abs_predicted_energies = abs_predicted  # Directly the predicted energies

    for i in range(len(tot_labels)):  # Iterate over samples
        valid_indices = tot_labels[i] != -1  # Indices where labels are not -1
        
        # Compute Mean Absolute Error (MAE) for energy reconstruction, excluding -1 labels
        abs_mape = np.sum(abs_predicted_energies[i][valid_indices])/np.sum(tot_labels[i][valid_indices])

        abs_mae_metrics.append(abs_mape)

    # Optionally, print comparisons for the first few samples of the first few files
    if npz_file_idx < 3:  # Print for the first 3 files
        print(f"File: {npz_file}")
        for sample_idx in range(min(5, len(feats))):  # Print first 3 samples
            valid_indices = tot_labels[sample_idx] != -1
            print(f"\nSample {sample_idx}:")

            abs_predicted_formatted = [f'{value:.5f}' for value in abs_predicted_energies[sample_idx][valid_indices]]
            true_energies_formatted = [f'{value:.5f}' for value in tot_labels[sample_idx][valid_indices]]

            print(f"Absolutely Predicted Energies: \n[{', '.join(abs_predicted_formatted)}]")  # Join the formatted strings for printing
            print(f"True Energies: \n[{', '.join(true_energies_formatted)}]\n")

# Compute and print overall metrics
average_abs_mape = np.mean(abs_mae_metrics)

print(f"Average MAPE for Absolute Predictions (excluding -1 labels): {(average_abs_mape)*100}%")

13/13 [==============================] - 31s 2s/step
File: /data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test/chunk_5_test.npz

Sample 0:
Normalized Absolutely Predicted Energies: 
[0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 0.00000, 

/tmp/ipykernel_68719/385064438.py:19: RuntimeWarning: invalid value encountered in scalar divide
  abs_mape = np.sum(abs_predicted_energies_masked) / np.sum(tot_labels[i][valid_indices])
/tmp/ipykernel_68719/385064438.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  abs_mape = np.sum(abs_predicted_energies_masked) / np.sum(tot_labels[i][valid_indices])


15/15 [==============================] - 34s 2s/step
File: /data/mjovanovic/jets/processed_files/2000_events_w_fixed_hits/SavedNpz/test/chunk_1_test.npz

Sample 0:
Normalized Absolutely Predicted Energies: 
[94.02119, 93.80322, 93.71712, 93.71543, 93.69862, 93.99014, 93.99033, 93.98969, 93.98621, 93.97604, 93.97353, 93.97137, 93.96576, 93.96327, 93.96075, 93.95522, 93.74020, 93.95359, 93.95096, 93.94772, 93.92828, 93.92342, 93.90674, 93.89771, 93.88819, 93.88799, 93.86697, 93.85516, 93.80686, 93.78129, 93.78665, 93.74928, 93.72964, 93.72214, 93.70236, 93.70628, 93.68790, 93.69090, 93.67126, 93.57953, 93.67783, 93.65656, 93.56715, 93.65445, 93.64004, 93.47841, 93.63466, 93.61819, 93.46506, 93.62777, 93.59422, 93.62827, 93.55891, 93.43803, 93.53082, 92.93570, 93.44940, 92.74544, 93.41322, 93.57120, 93.48244, 93.39998, 93.54062, 93.50102, 93.53838, 93.51080, 93.53064, 93.50302, 93.52235, 93.49840, 93.34885, 93.50957, 93.48677, 93.41321, 93.48815, 93.47327, 93.48585, 93.46343, 93.45164, 93